In [15]:
import pandas as pd
import regex as re
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

In [16]:
### Probamos  gaussiana 

In [17]:
def preprocess_dataframe(df):

  df.fillna(value=0, inplace=True) #Reemplazamos NAN por 0, ya que NAN rompe a Tensorflow

  #Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
  #Estos particularmente son whitespace, coma y parentesis por ejemplo.
  df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                    'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                    'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

  df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
  df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) 
    #corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. 
    #Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

  df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                  format='%m/%d/%Y')
  df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df = df[df['Opportunity_ID'] != 9773] 
 #Hardcodeo este filtrado porque el id 9773 tiene mal cargada
 #la fecha de delivery end, dando una diferencia de 200 anios"

  #Pongo .loc porque pandas me jode con warnings que son falsos positivos de slice copy"
  #Gracias Pandas!"

  #Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
  #estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
  #una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
  #la chance de completar la operacion.
  df['Delta_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
  df.loc[:, 'Delta_Time'] = df['Delta_Time'].dt.days
  df['Delta_Time'] = df['Delta_Time'].replace({np.nan:10.0}) 
  #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%,
  #y el analisis de los datos da que el 60% es maso a los 10 dias.
  #Asi no jodo el resto de los datos
  df['Delta_Time'] = df.groupby('Opportunity_ID')['Delta_Time'].transform('max')

  #Pasamos todo a dolares
  currency_conversion = {'AUD':0.707612, 'EUR':1.131064, 'GBP':1.318055, 'JPY':0.008987, 'USD':1.0}
  df['Total_Taxable_Amount_Currency'] = df[['Total_Taxable_Amount_Currency']].replace(currency_conversion)
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount_Currency'] * df['Total_Taxable_Amount']

  #Modifico la columna Brand para que en vez de decir que marca es, solo diga
  #si tiene o no marca. Es importante aclarar que verificamos que siempre que una oportunidad
  #tiene un producto con marca entonces todos sus productos tienen marca. Esto se cumple
  #tanto en el set de entrenamiento como en el de test, por lo tanto al hacer drop_duplicates
  #no nos va a pasar nunca el caso donde nos pudieramos quedar con una entrada de producto
  #sin marca mientras que algun otro producto si tuviera, ya que confirmamos que o todos tienen
  #marca o ninguno tiene.
  df.loc[df['Brand'] == 'None', 'Brand'] = 'No'
  df.loc[df['Brand'] != 'No', 'Brand'] = 'Yes'

  #Agrego una columna que indica si tiene o no numero de contrato
  df.loc[:, 'Sales_Contract_No'][df['Sales_Contract_No'] != 'None'] = 'Yes'
  df.loc[:, 'Sales_Contract_No'][df['Sales_Contract_No'] == 'None'] = 'No'
  df.rename(columns={'Sales_Contract_No':'Has_Contract_Number'}, inplace=True)

  #Agrego una columna que indique la cantidad de productos que tiene esa
  #oportunidad
  df['Product'] = 1
  df['Product_Amount'] = df.groupby('Opportunity_ID')['Product'].transform(lambda x: x.sum())

  #Agrego una columna que indica si el owner de la cuenta es el mismo que el de la oportunidad
  #o no
  df['Same_Owner'] = (df['Account_Owner'] == df['Opportunity_Owner'])
  df['Same_Owner'] = df['Same_Owner'].replace({False:'No', True:'Yes'})

  #Agrego una columna que indica si tiene o no fecha de expiracion
  df['Quote_Expiry_Date'] = (df['Quote_Expiry_Date'] != 'NaT')
  df.rename(columns={'Quote_Expiry_Date':'Has_Expiry_Date'}, inplace=True)
  df['Has_Expiry_Date'] = df['Has_Expiry_Date'].replace({True:'Yes',False:'No'})

  #Reemplazo las 4 columnas de aprobacion por solo 2 columnas que indiquen si tuvo la aprobacion
  #de delivery y burocratica o no. Recalco que si nunca la necesito seria equivalente a si
  #la necesito y la consiguio.
  df['Delivery_Approved'] = df['Pricing_Delivery_Terms_Quote_Appr'] + df['Pricing_Delivery_Terms_Approved']
  df['Delivery_Approved'] = df['Delivery_Approved'].replace({0:1, 1:0, 2:1})
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_0_Approval'] + df['Bureaucratic_Code_0_Approved']
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_Approved'].replace({0:1, 1:0, 2:1})
  df['Approved'] = df['Delivery_Approved'] & df['Bureaucratic_Code_Approved']

  #Cambio TRF por una columna que es el valor medio de los TRF de la oportunidad
  df["TRF"] = df.groupby("Opportunity_ID")["TRF"].transform("mean")

  def combineProducts(x):
    products = ""
    added = []
    for product in x:
      product = re.findall('\d+', product)[0]
      if added.count(product) == 0:
        products += (product)
        added.append(product)
    return products

  #Junto todos los productos en una sola entrada
  df['Products'] = df.groupby('Opportunity_ID')['Product_Family'].transform(combineProducts)

  #Pruebo volar duplicados, solo cambia el producto. Si el producto no importa
  #entonces volar duplicados no deberia importar. Obviamente vuelo el producto en el que
  #quede tambien.
  df.drop_duplicates('Opportunity_Name',inplace=True)
  df.drop(columns=['Product_Name','Product_Family','Opportunity_Name'],inplace=True)

  #Normalizo las columnas numericas
  normalized_columns = ['ASP_converted','TRF','Total_Taxable_Amount', 'Product_Amount', 'Delta_Time']
  for column in normalized_columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

  #Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
  #Algunas columnas borradas son porque pienso que no tienen incidencia, ir viendo.
  #TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
  #o no tiene numero de contrato. Por ahora no lo meto como input.
  #TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
  #y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
  #o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
  #como input.
  df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                  'Prod_Category_A', 'ID', 'Opportunity_ID', 
                   'Actual_Delivery_Date'],inplace=True)

  #Drop columnas que quiza podamos usar pero por ahora no las uso
  df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                  'Last_Modified_Date',
                  'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                  'Month',
                  'Delivery_Year',
                  'Price','ASP','ASP_Currency','Total_Amount_Currency',
                  'Total_Amount','Total_Taxable_Amount_Currency','Currency',
                   'Product_Category_B','Last_Modified_By', 'Account_Owner',
                   'Opportunity_Owner','Account_Name','Product_Type','Size',
                   'Billing_Country', 'Pricing_Delivery_Terms_Quote_Appr',
                   'Pricing_Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
                   'Bureaucratic_Code_0_Approved',
                   'Approved', 'Has_Contract_Number','Territory']
                   ,inplace=True)
  
  #Definimos que tipo de feature es cada columna

  #Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
  #si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
  #Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
  #Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
  #registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

  #Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
  #al azar.

  return df

In [18]:
csv = pd.read_csv("Train_TP2_Datos_2020-2C.csv") 
df = preprocess_dataframe(csv).copy()

/home/riedel/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/riedel/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/riedel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [19]:
#Setup para el Label Encoding

le = preprocessing.LabelEncoder()
categorical_columns = ["Region","Bureaucratic_Code","Source","Account_Type",
                       "Opportunity_Type","Quote_Type","Delivery_Terms","Brand",
                       "Has_Expiry_Date","Delivery_Quarter","Same_Owner"]

no_enc_df = df.copy() #Creo este aux para mantener el encoding para las predicciones

In [20]:
#Le aplico la codificacion a las columnas categoricas

for feature in categorical_columns:
  le.fit(df[feature]) #Calcula los codigos de cada valor de la serie que recibe
  df[feature] = le.transform(df[feature]) #Le asigno al df los valores codificados

df.head(10)

,Region,Bureaucratic_Code,Source,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Brand,Has_Expiry_Date,ASP_converted,...,TRF,Total_Taxable_Amount,Stage,Delta_Time,Product,Product_Amount,Same_Owner,Delivery_Approved,Bureaucratic_Code_Approved,Products
0,2,4,0,2,1,1,2,0,1,0.215056,...,0.482815,0.547199,0,0.762596,1,-0.427738,0,1,1,77
1,2,4,0,2,1,1,2,0,1,0.227577,...,-0.204252,-0.172044,1,-0.430272,1,-0.427738,0,1,1,77
2,1,4,12,4,1,1,4,0,1,0.095303,...,-0.204252,-0.168476,1,-0.471406,1,-0.427738,0,1,1,81
3,1,5,4,4,11,1,1,1,0,0.150657,...,0.757642,0.724658,0,0.721462,1,-0.427738,1,0,0,209
4,1,5,4,4,11,1,1,1,1,0.150657,...,1.513417,1.447064,0,0.083895,1,-0.427738,1,0,0,209
5,1,5,4,4,11,1,1,1,1,0.150657,...,1.719537,1.627332,0,0.721462,1,-0.427738,1,0,0,209
6,1,5,4,4,11,1,4,0,0,-0.015405,...,0.276695,0.145012,0,1.359030,1,-0.427738,1,0,0,164
7,1,4,12,4,1,1,4,0,1,0.095303,...,-0.204252,-0.172514,1,-0.471406,1,-0.427738,0,1,1,143
8,3,5,0,2,22,1,4,0,0,-0.436097,...,-0.204252,-0.178169,0,-0.471406,1,1.934778,1,0,1,642262377
13,3,5,0,2,22,1,4,0,0,-0.436097,...,-0.204252,-0.178072,0,-0.471406,1,1.934778,1,0,1,642262377


In [21]:
X = df.copy()

In [22]:
y = X['Stage'].copy()
X.drop(columns=['Stage'],inplace=True)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = GaussianNB()
y_pred = model.fit(X_train, y_train).predict_proba(X_test)
log_loss = metrics.log_loss(y_test, y_pred)
log_loss

5.506669844358751

### Exportarmos el csv entrenando con  100

In [24]:
X = df
y = X['Stage'].copy()
X.drop(columns=['Stage'],inplace=True)
model = GaussianNB()
model.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
frio_test_df = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
frio_test_df['Stage'] = 'Closed Won'  #por el preproces de data frame. 
aux_df = frio_test_df[['Opportunity_ID']] #Esta columna la vuela el preprocesado sino
aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True)

frio_test_df = preprocess_dataframe(frio_test_df)

for feature in categorical_columns:
  le.fit(no_enc_df[feature])
  frio_test_df[feature] = le.transform(frio_test_df[feature])
frio_test_df.sample(5)

/home/riedel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/riedel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/riedel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Region,Bureaucratic_Code,Source,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Brand,Has_Expiry_Date,ASP_converted,...,TRF,Total_Taxable_Amount,Stage,Delta_Time,Product,Product_Amount,Same_Owner,Delivery_Approved,Bureaucratic_Code_Approved,Products
940,2,4,2,2,1,1,2,1,1,-0.087932,...,-0.058329,-0.030253,1,0.855562,1,0.836994,1,1,1,117161132
1732,1,4,0,2,22,1,4,0,0,0.110593,...,-0.068611,-0.086368,1,-0.444175,1,3.887158,1,1,1,162168
2149,2,4,0,4,1,1,2,0,1,-0.032766,...,-0.058329,-0.059312,1,1.226915,1,0.226962,1,1,1,160139
1037,1,4,0,0,22,1,4,0,0,0.047195,...,-0.085748,-0.079597,1,-0.550276,1,-0.383071,1,1,1,168
755,1,4,0,2,22,1,6,0,0,-0.001574,...,-0.085748,-0.085494,1,1.757420,1,-0.383071,1,1,1,133


In [26]:
target = frio_test_df["Stage"].copy()
frio_test_df.drop(columns=["Stage"],inplace=True)

features = frio_test_df.values

predictions = model.predict_proba(features)

aux_df['Target'] = predictions[:,1]

aux_df.to_csv('OutPut/GausNB.csv', index=False)
aux_df.sample(3)
## Esto esta dando un valor absurdo.

,Opportunity_ID,Target
2415,12277,0.000024
2005,12033,0.000024
1798,11912,0.000024
